Проверяем создание `OfferMessage` из `UnifiedOffer`

In [1]:
import sys
sys.path.append('../../price_estimator/price-estimator-rest-api/')
sys.path.append('../../price_estimator/price-estimator-rest-api/proto')
sys.path.append('../../price_estimator/price-estimator-rest-api/api')
sys.path.append('../../price_estimator/price-estimator-rest-api/test')

In [2]:
from realty.offer import common_pb2, RealtySchema_pb2
from realty.offer import unified_offer_pb2
from proto.realty import model_pb2

In [3]:
from create_request_offer import test_one_request_api

In [4]:
import pandas as pd

In [5]:
import yt.wrapper as yt

In [6]:
yt.config["proxy"]["url"] = "hahn"

# Part 1. snapshot

In [52]:
# выбираем активные объявления из сломанного снэпшота
# см https://yql.yandex-team.ru/Operations/YFhnRL94hiHQKaX7ZZJbW3F-C68pVXmP3XUCM6PKQpY=

In [8]:
table_path = "//home/verticals/realty/price_estimator/VSQUALITY-3920/active_snapshot"
sampling_rate = 0.001

In [9]:
iter_table = yt.read_table(table_path,
                           format=yt.YsonFormat(encoding=None),
                           table_reader={
                               "sampling_seed": 0,
                               "sampling_rate": sampling_rate
                           })

**OfferMessage**

**offer_id** = 2707628339876167681

**predicted_price** = 25000

In [15]:
# for i,row_old in enumerate(iter_table):
#     break

# request_old = RealtySchema_pb2.OfferMessage()
# request_old.ParseFromString(row_old[b'offer'])
# request_old.offer_id, request_old.offerTypeInt, request_old.categoryTypeInt
# >>> ('2707628339876167681', 2, 2)

# test_one_request_api(
#     request_old,
#     hostname='realty-price-estimator.vrts-slb.test.vertis.yandex.net',
#     path='get_price_offer'
# )
# >>> predicted_price {
#   min: 22000
#   max: 27000
#   value: 25000
#   q05: 13000
#   q25: 17000
#   q75: 38000
#   q95: 50000
# }

# Part 2. holocron: broker

### auxiliary data

In [13]:
# получаем из базы домов число квартир по каждому адресу
# см https://yql.yandex-team.ru/Operations/YGGmc9K3DKODX-MM_qsaplst7hD3Mg4ycbfTJO4WeUQ=

df_buildings_flat_count = pd.read_csv("./buildings_flat_count.tsv", sep="\t", skiprows=1, names=["building_id", "unified_address", "flats_count"])

buildings_flat_count = df_buildings_flat_count.set_index("unified_address")["flats_count"].to_dict()

### func

In [28]:
# Пишем функцию которая из полей в холокроне брокера создает OfferMessage для передачи в price-estimator
def create_offer_message(offer_id, unified_offer, area, rooms_offered, buildings_flat_count):
    """На основе UnifiedOffer и доп.полей создает OfferMessage"""
    request = RealtySchema_pb2.OfferMessage()

    request.offer_id = offer_id
    request.apartmentInfo.apartment = ""
    request.apartmentInfo.studio = unified_offer.apartment.studio
    try:
        request.apartmentInfo.floors.append(unified_offer.apartment.highrise_apartment_info.floors[0])
    except:
        pass
    request.apartmentInfo.ceilingHeight = unified_offer.apartment.general_apartment_info.ceiling_height.value
    # request.apartmentInfo.roomsOffered = unified_offer.room.rooms_offered
    request.apartmentInfo.roomsOffered = rooms_offered
    request.apartmentInfo.rooms = unified_offer.apartment.rooms_total
    request.apartmentInfo.balconyInt = unified_offer.apartment.living_info.balcony
    request.apartmentInfo.renovationInt = unified_offer.apartment.general_apartment_info.renovation
    request.apartmentInfo.flatTypeInt = unified_offer.apartment.flat_type
    # request.transaction.area.value = unified_offer.transaction_info.sq_m_price_area
    request.transaction.area.value = area
    request.area.value = unified_offer.apartment.apartment_area
    request.offerTypeInt = 2 if unified_offer.WhichOneof("type") == "rent" else 1
    request.categoryTypeInt = 2 if unified_offer.WhichOneof("category") == "apartment" else 1

    request.buildingInfo.floorsTotal = unified_offer.apartment.building_info.floors_total.value
    request.buildingInfo.buildingId = unified_offer.apartment.building_info.building_id.value
    request.buildingInfo.buildYear = unified_offer.apartment.building_info.built_year.value
    request.buildingInfo.buildingSeriesId = unified_offer.apartment.building_info.building_series.id
    request.buildingInfo.expectDemolition = unified_offer.apartment.building_info.expect_demolition.value
    request.buildingInfo.parkingTypeInt = unified_offer.apartment.building_info.parking_info.type
    try:
        request.buildingInfo.flatsCount = int(buildings_flat_count[unified_offer.location.unified_address.unified_oneline])
    except:
        pass
    request.buildingInfo.site_id = unified_offer.apartment.building_info.site_id
    # request.buildingInfo.has_elevator = 

    # request.lotInfo.lotArea.unitInt = 
    # request.lotInfo.lotArea.value = 
    # request.lotInfo.lotTypeInt = 

    # request.houseInfo.pmg = 
    # request.houseInfo.houseTypeInt = 
    request.houseInfo.livingSpace = unified_offer.apartment.living_space
    request.houseInfo.kitchenSpace = unified_offer.apartment.kitchen_space

    try:
        request.location.localityName = [
            c.value for c in unified_offer.location.unified_address.component
            if c.region_type ==  model_pb2.RegionType.Value('CITY')
        ][0]
    except:
        pass
    request.location.subjectFederationId = unified_offer.location.subject_federation_geoid
    # request.location.regionGraphId = 
    request.location.geocoderAddress = unified_offer.location.unified_address.unified_oneline
    request.location.geocoderPoint.latitude = unified_offer.location.unified_coordinates.latitude
    request.location.geocoderPoint.longitude = unified_offer.location.unified_coordinates.longitude
    
    return request

### one offer

In [17]:
# записываем в //home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16
# только один оффер 2707628339876167681
# INSERT INTO
#     hahn.`//home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16`
# SELECT
#     *
# FROM hahn.`home/verticals/broker/prod/warehouse/holocron/realty/full/eod/1d/2021-03-16` VIEW raw
# WHERE id == '2707628339876167681'
table_path = "//home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16"

In [18]:
iter_table = yt.read_table(table_path,
                           format=yt.YsonFormat(encoding=None))

In [ ]:
for row2 in iter_table:
    break

In [ ]:
request = create_offer_message(
    offer_id=row2[b'offer_id'].decode(),
    raw_unified_offer=row2[b'raw_offer'],
    area=row2[b'area'],
    rooms_offered=row2[b'rooms_offered'],
    buildings_flat_count=buildings_flat_count
)

**OfferMessage via UnifiedOffer**

**offer_id** = 2707628339876167681

**predicted_price** = 25000

In [312]:
test_one_request_api(
    request,
    hostname='realty-price-estimator.vrts-slb.test.vertis.yandex.net',
    path='get_price_offer'
)

predicted_price {
  min: 22000
  max: 27000
  value: 25000
  q05: 13000
  q25: 17000
  q75: 38000
  q95: 50000
}

In [306]:
# >>> predicted_price {
#   min: 22000
#   max: 27000
#   value: 25000
#   q05: 13000
#   q25: 17000
#   q75: 38000
#   q95: 50000
# }

для уникального объявления совпадает. посчитаем разницу на выборке

## sample N=100

In [10]:
table_path = "//home/verticals/realty/price_estimator/VSQUALITY-3920/active_snapshot"
sampling_rate = 0.001

iter_table = yt.read_table(table_path,
                           format=yt.YsonFormat(encoding=None),
                           table_reader={
                               "sampling_seed": 0,
                               "sampling_rate": sampling_rate
                           })

result = {}
N = 100
for i,row_old in enumerate(iter_table):
    request_old = RealtySchema_pb2.OfferMessage()
    request_old.ParseFromString(row_old[b'offer'])
    request_old.offer_id, request_old.offerTypeInt, request_old.categoryTypeInt
    response = test_one_request_api(
        request_old,
        hostname='realty-price-estimator.vrts-slb.test.vertis.yandex.net',
        path='get_price_offer'
    )
    result[request_old.offer_id] = response
    if i >= N:
        break

### prepare data

In [14]:
yql_query = f"""
INSERT INTO
    hahn.`//home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16`
SELECT
    *
FROM hahn.`home/verticals/broker/prod/warehouse/holocron/realty/full/eod/1d/2021-03-16` VIEW raw
WHERE id IN({str(set(result))[1:-1]})
"""
print(yql_query)


INSERT INTO
    hahn.`//home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16`
SELECT
    *
FROM hahn.`home/verticals/broker/prod/warehouse/holocron/realty/full/eod/1d/2021-03-16` VIEW raw
WHERE id IN('4772780070845500422', '8037529677593313793', '6302897158521575542', '4772780070841005050', '3160692721945911552', '4759594191718980865', '5461318868036607214', '2265367850308131749', '5361108677139925804', '5361109690554612902', '8028692506058197799', '5361108782780569473', '4130104605715456513', '5361110432511384302', '4609934776453502977', '2670175299082632167', '7568607406433874983', '1533960538893573888', '5361113045288974410', '6302897158524344183', '4772780070840890760', '4384727186725414395', '4772780070838887881', '377153652876651264', '2078877692386170881', '219353371354203206', '2265367795365876266', '6836428642663473729', '2789325247447193088', '4772780070840885893', '5205645657346572344', '8606660881716015360', '5206250133453018584', '1564264441921874944',

In [29]:
table_path = "//home/verticals/realty/price_estimator/VSQUALITY-3920/holocron_2021_03_16"

iter_table = yt.read_table(table_path,
                           format=yt.YsonFormat(encoding=None))

result2 = {}
for row2 in iter_table:
    offer_id=row2[b"id"].decode()
    request = create_offer_message(
        offer_id=offer_id,
        raw_unified_offer=row2[b'raw_offer'],
        area=row2[b'area'],
        rooms_offered=row2[b'rooms_offered'],
        buildings_flat_count=buildings_flat_count
    )
    response = test_one_request_api(
        request,
        hostname='realty-price-estimator.vrts-slb.test.vertis.yandex.net',
        path='get_price_offer'
    )
    result2[offer_id] = response

In [32]:
len(result), len(result2)

(59, 11)

In [35]:
for key in result2:
    print(key, result[key].predicted_price.value, result2[key].predicted_price.value)

784634875377120512 50000 49000
4772780070843534405 6077000 6821000
4772780070840885893 4001000 4681000
5361109552164067531 11200000 11774000
1608630376234427392 7066000 8064000
1533960538893573888 36000 29000
219353371354203206 3285000 3265000
4772780070838887881 5156000 6003000
6302897158522532449 6105000 6238000
8606660881716015360 1557000 1543000
1862529429328601089 59000 59000


### Вывод
разница есть, в 6 из 11 случаев оценка через UnifiedOffer больше, чем через OfferMessage

- Для выгрузки модерации это не критично, тк прогнозы на проде в среднем лучше чем в выгрузке
- Однако при переходе на UnifiedOffer полностью нужно проверить соответствие полей в моделях